# Interactive Analytics
<br>
## Task
Run interactive queries on the data and answer some analytical questions

## Data
* use two datasets about influencers
* the first dataset contains basic information about each influencer
* the second dataset contains posting history for each influncer for the past 6 months

## Questions:
1) Count number of distinct languages used in the dataset

2) List all interests in descending order depending on its frequency

3) How many influencers interested in Technology using english language (en) published a post in july and did not publish a post in june

4) For each influencer add a new column n_posts with number of posts he published in its history 

5) Find top 10 influencers that posted the most posts in their history

6) Compute entropy of the n_posts column for each interest

## Documentation
<br>
* Pyspark documentation of DataFrame API is <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html">here</a>

* Prezentation slides are accessed <a target="_blank" href = "https://docs.google.com/presentation/d/1XNKIfE5Atj_Mzse0wjmbwLecmVs2YkWm9cqOLqDVWPo/edit?usp=sharing">here</a>

### Import functions and modules

In [4]:
from pyspark.sql.functions import count, col, size, greatest, collect_list, when, explode, sum, rank, dense_rank, desc, row_number, avg, pandas_udf, PandasUDFType, array_contains, array_sort, lit, dayofweek
from pyspark.sql import Window
from pyspark.sql.types import DateType, DoubleType, StructType, StructField, DecimalType, LongType, StringType

import scipy.stats

### Load the data

In [6]:
infls = spark.table('mlprague.influencers')

posts_history = spark.table('mlprague.infl_posting_history')

### Explore the data

hint
* check the schema call df.printSchema()
* check the row count - call df.count()
* see some rows - call df.show(), or display(df)

In [8]:
infls.printSchema()

In [9]:
display(infls)

In [10]:
infls.count()

### Count number of distinct languages used in the dataset (*)

hint
* use <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.explode">explode</a> and <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct">distinct</a>

In [12]:
(
  infls
  .withColumn('language', explode('languages'))
  .select('language')
  .distinct()
  .count()
)

### List all interests in descending order depending on its frequency (\* \*)

hint:
* use explode on the array
* use groupBy with some aggregation function (slide 25 in the presentation might be useful for aggregations)
* use <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy">orderBy</a>

In [14]:
display(
  infls
  .withColumn('interest', explode('interests'))
  .groupBy('interest')
  .agg(
    count('*').alias('interest_frequency')
  )
  .orderBy(desc('interest_frequency'))
)

### How many influencers interested in Technology using english language (en) published a post in july and did not publish a post in june (\* \* \*)

hint
* Possible way how to do it:
 * join influencers with their posting history
 * create 2 dataframes - in the first one select influencers that published in july, in the second select influencers that published in june
 * apply filters for interest and language
 * do a left anti join ('left_anti')
 * deduplicate the result
* <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join">joins</a> in the docs
* slide 20 in the presentation might be useful for joins
* <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropDuplicates">dropDuplicates</a> in the docs

In [16]:
july = (
  infls
  .join(posts_history, 'influencer_id')
  .filter(array_contains(col('languages'), 'en'))
  .filter(array_contains(col('interests'), 'Technology'))
  .filter(col('post_date').between('2018-07-01', '2018-07-31'))
)

june = (
  infls
  .join(posts_history, 'influencer_id')
  .filter(array_contains(col('languages'), 'en'))
  .filter(array_contains(col('interests'), 'Technology'))
  .filter(col('post_date').between('2018-06-01', '2018-06-30'))
)

july.join(june, 'influencer_id', 'left_anti').dropDuplicates(['influencer_id']).count()


### For each influencer add a new column n_posts with number of posts he published in its history (\* \*)

hint
* This question is prerequisite for the two next questions
* Possible way how to do it:
 * aggregate the posting history and join the result to influencers

In [18]:
n_posts = (
  posts_history
  .groupBy('influencer_id')
  .agg(
    count('*').alias('n_posts')
  )
)

infls_with_n_posts = (
  infls
  .join(n_posts, 'influencer_id')
  
)

In [19]:
display(infls_with_n_posts)

### Find top 10 influencers that posted the most posts in their history (*)

hint
* this question depends on the result from the previous question
* orderBy the previous result

In [21]:
display(
  infls_with_n_posts
  .orderBy(desc('n_posts'))
  .limit(10)
)

### Compute entropy of the n_posts column for each interest (\* \* \* \* \*)

hint
* This question depends on results from the previous 2 questions
* There are at least 2 ways how you can solve this (choose which way you prefer)
* Way 1:
 * Using higer order functions (slide 55 in the presentation might be useful)
 * first compute probability for each interest (for each interest get an array with probabilities of n_posts col)
 * (you might want to groupBy interest and n_posts than use a window and finaly groupBy again by interest and use collect_list for final aggregation)
 * use AGGREGATE in SQL expression to compute the entropy from the array
* Way 2:
  * Using Pandas grouped_map UDF (slide 40 in the presentation might be useful)
  * Define schema for the udf
  * Implement the udf (to compute the probabilities you can use value_counts() of Pandas.Series and divide by number of rows using len(pandas dataframe))
  * Use scipy.stats.entropy for the entropy calculation
  
Note
* For entropy calculation use the same definition as is used in scipy: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html

$$S = - \Sigma_{i} \left(p\left(i\right) * \log\left(p\left(i\right)\right)\right)$$

* for probability computation you can use this approach: compute the frequency of each value and divide by the total number of values
 * example: suppose we have this array [10, 10, 11, 12], the probability of 10 is p(10) = 2/4 = 0.5, the probability of 11 is p(11) = 1/4 = 0.25

In [23]:
# Using Higher order functions:

interests_with_probability = (
  infls_with_n_posts
  .select('n_posts', explode('interests').alias('interest'))
  .select(col('n_posts').cast(DoubleType()).alias('n_posts'), 'interest')
  .groupBy('interest', 'n_posts')
  .agg(count('*').alias('frequency'))
  .withColumn('n_posts_total', sum('frequency').over(Window.partitionBy('interest')))
  .withColumn('n_posts_probability', col('frequency') / col('n_posts_total'))
  .groupBy('interest')
  .agg(
    collect_list(col('n_posts_probability')).alias('n_posts')
  )
)

In [24]:
display(interests_with_probability)

In [25]:
result = interests_with_probability.selectExpr(
  'interest',
  'AGGREGATE(n_posts, CAST(0 AS double), (buffer, value) -> (buffer - value * Log(value))) AS n_posts_entropy'
).orderBy('n_posts_entropy')

In [26]:
display(result)

In [27]:
# using Pandas GROUPED_MAP UDF:

interests_with_n_posts = (
  infls_with_n_posts
  .select('n_posts', explode('interests').alias('interest'))
)


schema = StructType(
  [
    StructField('n_posts', LongType()),
    StructField('interest', StringType()),
    StructField('entropy', DoubleType())
  ]
)

@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def udf_entropy(pdf):
    p_data= pdf['n_posts'].value_counts() / len(pdf) # calculates the probabilities
    pdf['entropy'] = scipy.stats.entropy(p_data)
    return pdf

In [28]:
result = (
  interests_with_n_posts
  .groupBy('interest')
  .apply(udf_entropy)
  .select('interest', 'entropy')
  .dropDuplicates()
  .orderBy('entropy')
)

In [29]:
display(result)

### You may want to take a look at the distribution for given interests

In [31]:
display(
  interests_with_n_posts
  .filter(col('interest') == 'Technology')
  .groupBy('n_posts')
  .agg(
    count('*')
  )
  .orderBy('n_posts')
)

In [32]:
display(
  interests_with_n_posts
  .filter(col('interest') == 'Hobbies and activities')
  .groupBy('n_posts')
  .agg(
    count('*')
  )
  .orderBy('n_posts')
)